<div class="alert alert-block alert-warning">
Simplest way <br> 
1) install anaconda<br>
2) conda env create -f rebri_env.yml
</div>

# UEF spectra handling 

<div class="alert alert-block alert-success">

<li>Soft for conjugated scanning-table and mobile spectrometer control was written by Kudarenko
<li>Transfer optical system (made by Kudarenko) from MSU to UEF, CVD processes, spectra collection were done by Malykhin
<li>Final engineering of the optical system at the UEF - Itkonen
<li>ML, Supervisor, inspirer - Ismagilov

</div>


## Part 1a. Glue 2 files to get XY_WL dataset for further $T_{e}$ or specific OES line distribution calculations

In [1]:
# Author: Rinat R. Ismagilov <ismagil@polly.phys.msu.ru>
#
# License: GNU General Public License v3.0

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time 
%matplotlib inline

pd.options.display.max_rows = 10
# pd.options.display.float_format = '{:.1f}'.format


In [2]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [3]:
# nice practice

%load_ext version_information
# pip install version_information

%version_information numpy, pandas, matplotlib, sklearn, version_information

C:\Anaconda3\envs\rebri_env\lib\site-packages\sklearn\utils\fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


Software versions
Python 3.5.6 64bit [MSC v.1900 64 bit (AMD64)]
IPython 6.5.0
OS Windows 10 10.0.18362 SP0
numpy 1.10.1
pandas 0.20.3
matplotlib 3.0.0
sklearn 0.20.0
version_information 1.0.3
Thu Oct 10 15:56:46 2019 RTZ 2 (ceia)

In [4]:
from _myconfig import *
raw_Kud, raw_Mal, my_fname, rastr, tsync

('spec_scan_2019_10_10__13-18-06.txt',
 'MPMA_20191010131801.txt',
 '__118_13_18_',
 (55, 99),
 5)

## Uploading raw data (data engineering)

In [5]:
# raw_Kud = "spec_scan_2019_10_01__14-19-20.txt"
# raw_Mal = "MPMA_20191001141917.txt"
# my_fname = "__114_14_19_"
# rastr = (67,113) #kud_df

kud_df = pd.read_csv(raw_Kud, delimiter= " ", skiprows = 3, header=None, names = ["x","y","scan_time"], 
                       dtype = {'x':"float64", 'y':"float64", "scan_time":np.str})

dti = pd.to_datetime(kud_df['scan_time'])
dti -= dti[0] #set starting time to 0
dti /= np.timedelta64(1, 's') # convert to seconds
kud_df['time'] = dti


print(kud_df.dtypes, kud_df.shape)

x            float64
y            float64
scan_time     object
time         float64
dtype: object (5600, 4)


In [6]:
mal_df = pd.read_csv(raw_Mal, delimiter= "\t", header=None, dtype = np.str)
mal_df = mal_df.iloc[:,:1025].copy()
for i in np.arange(mal_df.shape[1]):
        mal_df[i] = mal_df[i].str.replace(',','.')
wl = 10*np.array(mal_df.iloc[:1,1:].values,dtype='float64')
intens = np.array(mal_df.iloc[1:,1:].values,dtype='float64')
t = np.array(mal_df.iloc[1:,:1].values,dtype='float64')

mal_df=pd.DataFrame(intens, columns = wl[0])
mal_df['time'] = t #the same as for "kud", we are going to join (outer)
print(mal_df.dtypes, mal_df.shape)

1922.31    float64
1929.96    float64
1937.62    float64
1945.27    float64
1952.92    float64
            ...   
9538.85    float64
9546.09    float64
9553.33    float64
9560.57    float64
time       float64
Length: 1025, dtype: object (11742, 1025)


In [7]:
# display('df1', 'df5', "pd.merge(df1, df5)")
display('mal_df', 'kud_df')

,1922.31,1929.96,1937.62,1945.27,1952.92,1960.58,1968.23,1975.8799999999999,1983.5300000000002,1991.19,...,9502.65,9509.89,9517.13,9524.37,9531.609999999999,9538.85,9546.09,9553.33,9560.57,time
0,27.281,-2.273,-11.367,27.281,61.383,-18.188,0.000,-25.008,-36.375,-70.477,...,604.932,782.804,1098.620,856.012,986.027,1037.530,945.713,1163.568,1097.596,0.00
1,72.750,25.008,-22.734,2.273,72.750,-4.547,-22.734,-43.195,-2.273,-97.758,...,691.351,767.251,1103.591,879.923,958.381,1104.611,1020.144,1290.043,928.114,0.25
2,11.367,13.641,-2.273,47.742,72.750,2.273,-4.547,-22.734,0.000,-47.742,...,729.160,689.490,1009.140,765.150,893.875,1050.946,976.361,1100.331,1008.820,0.50
3,68.203,45.469,-22.734,27.281,50.016,-34.102,11.367,-9.094,-11.367,-77.297,...,669.747,834.645,1078.735,841.665,884.660,1037.530,950.091,1159.352,1061.278,0.75
4,20.461,29.555,-22.734,-15.914,81.844,-11.367,-15.914,-18.188,-4.547,-79.571,...,691.351,782.804,1063.822,851.230,953.774,1019.641,954.470,1138.273,1024.961,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11737,325.103,302.368,256.900,188.696,256.900,190.970,275.087,250.079,204.610,145.501,...,24478.160,24697.203,25407.449,25422.121,25033.104,25772.773,26151.592,27107.760,26774.076,2934.25
11738,325.103,302.368,222.798,263.720,263.720,213.704,245.532,270.540,204.610,147.774,...,24548.375,24852.727,25462.133,25527.330,25774.928,25813.023,26663.855,27267.959,27064.617,2934.50
11739,279.634,311.462,220.524,177.329,247.806,215.977,256.900,275.087,211.431,143.227,...,24499.764,24510.574,25402.479,25484.291,25770.318,26184.209,26663.855,26993.932,27359.191,2934.75
11740,302.368,343.290,202.337,211.431,263.720,204.610,254.626,227.345,240.985,127.313,...,24521.369,25023.805,25407.449,25412.559,25650.523,25960.604,26475.588,27386.002,27399.545,2935.00


# <font color = 'red'>Change sync time below!!! (befor running script - "Run All")

In [8]:
mal_df['time'] = mal_df['time'] - 6.0 + tsync*0.25
#13-18-07.16 (kud) and 13-18-01 (mal filename) ->6.0
#e.g. 15-04-56.64 (kud_df first scan_time) and 15-04-52 (mal filename) ->4.50
mal_df

,1922.31,1929.96,1937.62,1945.27,1952.92,1960.58,1968.23,1975.8799999999999,1983.5300000000002,1991.19,...,9502.65,9509.89,9517.13,9524.37,9531.609999999999,9538.85,9546.09,9553.33,9560.57,time
0,27.281,-2.273,-11.367,27.281,61.383,-18.188,0.000,-25.008,-36.375,-70.477,...,604.932,782.804,1098.620,856.012,986.027,1037.530,945.713,1163.568,1097.596,-4.75
1,72.750,25.008,-22.734,2.273,72.750,-4.547,-22.734,-43.195,-2.273,-97.758,...,691.351,767.251,1103.591,879.923,958.381,1104.611,1020.144,1290.043,928.114,-4.50
2,11.367,13.641,-2.273,47.742,72.750,2.273,-4.547,-22.734,0.000,-47.742,...,729.160,689.490,1009.140,765.150,893.875,1050.946,976.361,1100.331,1008.820,-4.25
3,68.203,45.469,-22.734,27.281,50.016,-34.102,11.367,-9.094,-11.367,-77.297,...,669.747,834.645,1078.735,841.665,884.660,1037.530,950.091,1159.352,1061.278,-4.00
4,20.461,29.555,-22.734,-15.914,81.844,-11.367,-15.914,-18.188,-4.547,-79.571,...,691.351,782.804,1063.822,851.230,953.774,1019.641,954.470,1138.273,1024.961,-3.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11737,325.103,302.368,256.900,188.696,256.900,190.970,275.087,250.079,204.610,145.501,...,24478.160,24697.203,25407.449,25422.121,25033.104,25772.773,26151.592,27107.760,26774.076,2929.50
11738,325.103,302.368,222.798,263.720,263.720,213.704,245.532,270.540,204.610,147.774,...,24548.375,24852.727,25462.133,25527.330,25774.928,25813.023,26663.855,27267.959,27064.617,2929.75
11739,279.634,311.462,220.524,177.329,247.806,215.977,256.900,275.087,211.431,143.227,...,24499.764,24510.574,25402.479,25484.291,25770.318,26184.209,26663.855,26993.932,27359.191,2930.00
11740,302.368,343.290,202.337,211.431,263.720,204.610,254.626,227.345,240.985,127.313,...,24521.369,25023.805,25407.449,25412.559,25650.523,25960.604,26475.588,27386.002,27399.545,2930.25


In [9]:
kudmal_df = pd.merge(kud_df, mal_df, how='outer', on=['time']) # SQL-style
# pd.concat([df1, df4], axis=1, sort=False) # alternative
kudmal_df.shape, 7752+15000-1 # expecting 7752+15000-1 [one row 0==0 merge nicely] = 20441

((17340, 1028), 22751)

In [10]:
kudmal_df.sort_values(by='time', ascending=True, inplace=True)
kudmal_df

,x,y,scan_time,time,1922.31,1929.96,1937.62,1945.27,1952.92,1960.58,...,9495.41,9502.65,9509.89,9517.13,9524.37,9531.609999999999,9538.85,9546.09,9553.33,9560.57
5600,NaN,NaN,NaN,-4.75,27.281,-2.273,-11.367,27.281,61.383,-18.188,...,998.240,604.932,782.804,1098.620,856.012,986.027,1037.530,945.713,1163.568,1097.596
5601,NaN,NaN,NaN,-4.50,72.750,25.008,-22.734,2.273,72.750,-4.547,...,1071.973,691.351,767.251,1103.591,879.923,958.381,1104.611,1020.144,1290.043,928.114
5602,NaN,NaN,NaN,-4.25,11.367,13.641,-2.273,47.742,72.750,2.273,...,1009.583,729.160,689.490,1009.140,765.150,893.875,1050.946,976.361,1100.331,1008.820
5603,NaN,NaN,NaN,-4.00,68.203,45.469,-22.734,27.281,50.016,-34.102,...,975.552,669.747,834.645,1078.735,841.665,884.660,1037.530,950.091,1159.352,1061.278
5604,NaN,NaN,NaN,-3.75,20.461,29.555,-22.734,-15.914,81.844,-11.367,...,981.224,691.351,782.804,1063.822,851.230,953.774,1019.641,954.470,1138.273,1024.961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17335,NaN,NaN,NaN,2929.50,325.103,302.368,256.900,188.696,256.900,190.970,...,24950.316,24478.160,24697.203,25407.449,25422.121,25033.104,25772.773,26151.592,27107.760,26774.076
17336,NaN,NaN,NaN,2929.75,325.103,302.368,222.798,263.720,263.720,213.704,...,25347.344,24548.375,24852.727,25462.133,25527.330,25774.928,25813.023,26663.855,27267.959,27064.617
17337,NaN,NaN,NaN,2930.00,279.634,311.462,220.524,177.329,247.806,215.977,...,25069.424,24499.764,24510.574,25402.479,25484.291,25770.318,26184.209,26663.855,26993.932,27359.191
17338,NaN,NaN,NaN,2930.25,302.368,343.290,202.337,211.431,263.720,204.610,...,25523.170,24521.369,25023.805,25407.449,25412.559,25650.523,25960.604,26475.588,27386.002,27399.545


In [11]:
kudmal_df = kudmal_df[kudmal_df['time']>=0]
# kudmal_df.shape # 20.0/0.25 +1 = 30
kudmal_df, 20.0/0.25 + 1 # how many lines have we droped?

(         x    y        scan_time         time  1922.31  1929.96  1937.62  \
 0      0.0  0.0  13:18:07.156254     0.000000   25.008   50.016  -22.734   
 5619   NaN  NaN              NaN     0.250000   36.375   29.555  -13.641   
 5620   NaN  NaN              NaN     0.500000   43.195   15.914    0.000   
 1      0.0  1.0  13:18:07.676267     0.520013      NaN      NaN      NaN   
 5621   NaN  NaN              NaN     0.750000   63.657   40.922  -11.367   
 ...    ...  ...              ...          ...      ...      ...      ...   
 17335  NaN  NaN              NaN  2929.500000  325.103  302.368  256.900   
 17336  NaN  NaN              NaN  2929.750000  325.103  302.368  222.798   
 17337  NaN  NaN              NaN  2930.000000  279.634  311.462  220.524   
 17338  NaN  NaN              NaN  2930.250000  302.368  343.290  202.337   
 17339  NaN  NaN              NaN  2930.500000  286.454  322.829  245.532   
 
        1945.27  1952.92  1960.58    ...        9495.41    9502.65    9509

In [12]:
kudmal_df.reset_index(drop=True, inplace = True)

# There is no spectra for last coordinate point

In [13]:
kudmal_df[kudmal_df[1922.31].notnull()] # memorize index till which we have spectra (hamamatsu write only 15k spectra)

# kudmal_df.columns

,x,y,scan_time,time,1922.31,1929.96,1937.62,1945.27,1952.92,1960.58,...,9495.41,9502.65,9509.89,9517.13,9524.37,9531.609999999999,9538.85,9546.09,9553.33,9560.57
0,0.0,0.0,13:18:07.156254,0.00,25.008,50.016,-22.734,43.195,75.024,-6.820,...,1049.286,675.148,793.172,999.197,832.101,916.913,1019.641,993.874,1197.294,1000.749
1,NaN,NaN,NaN,0.25,36.375,29.555,-13.641,13.641,40.922,-9.094,...,918.834,626.537,824.277,1103.591,846.448,949.166,1055.418,1072.684,1184.647,920.044
2,NaN,NaN,NaN,0.50,43.195,15.914,0.000,61.383,70.477,-20.461,...,1020.927,675.148,694.674,1063.822,803.408,972.204,974.920,928.200,1142.489,1016.890
4,NaN,NaN,NaN,0.75,63.657,40.922,-11.367,20.461,93.211,-43.195,...,1003.911,680.549,839.829,1053.880,784.279,981.419,1162.749,1103.332,1142.489,1028.996
5,NaN,NaN,NaN,1.00,72.750,4.547,13.641,18.188,65.930,27.281,...,1106.004,707.555,813.909,1073.764,894.270,962.989,1104.611,1055.171,1323.769,1033.031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17316,NaN,NaN,NaN,2929.50,325.103,302.368,256.900,188.696,256.900,190.970,...,24950.316,24478.160,24697.203,25407.449,25422.121,25033.104,25772.773,26151.592,27107.760,26774.076
17317,NaN,NaN,NaN,2929.75,325.103,302.368,222.798,263.720,263.720,213.704,...,25347.344,24548.375,24852.727,25462.133,25527.330,25774.928,25813.023,26663.855,27267.959,27064.617
17318,NaN,NaN,NaN,2930.00,279.634,311.462,220.524,177.329,247.806,215.977,...,25069.424,24499.764,24510.574,25402.479,25484.291,25770.318,26184.209,26663.855,26993.932,27359.191
17319,NaN,NaN,NaN,2930.25,302.368,343.290,202.337,211.431,263.720,204.610,...,25523.170,24521.369,25023.805,25407.449,25412.559,25650.523,25960.604,26475.588,27386.002,27399.545


In [14]:
# till index 22170 we have spectra
kudmal_df[kudmal_df['x']>63]


,x,y,scan_time,time,1922.31,1929.96,1937.62,1945.27,1952.92,1960.58,...,9495.41,9502.65,9509.89,9517.13,9524.37,9531.609999999999,9538.85,9546.09,9553.33,9560.57


### since, only lines with x = 63, 64, 65, 66, 67 are absent (should be close to cathode)


In [15]:
col = ['x', 'y'] # down to kud's "time" (e.g. 0.512584) set current coordinates (x,y)
kudmal_df.loc[:,col] = kudmal_df.loc[:,col].ffill() # only x,y
# kudmal_df.loc[:,:] = kudmal_df.loc[:,:].ffill() # not only x,y - everything including cathode spectra
# pd.options.display.max_rows = 20
kudmal_df

# last steps does not 

C:\Anaconda3\envs\rebri_env\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,x,y,scan_time,time,1922.31,1929.96,1937.62,1945.27,1952.92,1960.58,...,9495.41,9502.65,9509.89,9517.13,9524.37,9531.609999999999,9538.85,9546.09,9553.33,9560.57
0,0.0,0.0,13:18:07.156254,0.000000,25.008,50.016,-22.734,43.195,75.024,-6.820,...,1049.286,675.148,793.172,999.197,832.101,916.913,1019.641,993.874,1197.294,1000.749
1,0.0,0.0,NaN,0.250000,36.375,29.555,-13.641,13.641,40.922,-9.094,...,918.834,626.537,824.277,1103.591,846.448,949.166,1055.418,1072.684,1184.647,920.044
2,0.0,0.0,NaN,0.500000,43.195,15.914,0.000,61.383,70.477,-20.461,...,1020.927,675.148,694.674,1063.822,803.408,972.204,974.920,928.200,1142.489,1016.890
3,0.0,1.0,13:18:07.676267,0.520013,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,1.0,NaN,0.750000,63.657,40.922,-11.367,20.461,93.211,-43.195,...,1003.911,680.549,839.829,1053.880,784.279,981.419,1162.749,1103.332,1142.489,1028.996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17316,55.0,99.0,NaN,2929.500000,325.103,302.368,256.900,188.696,256.900,190.970,...,24950.316,24478.160,24697.203,25407.449,25422.121,25033.104,25772.773,26151.592,27107.760,26774.076
17317,55.0,99.0,NaN,2929.750000,325.103,302.368,222.798,263.720,263.720,213.704,...,25347.344,24548.375,24852.727,25462.133,25527.330,25774.928,25813.023,26663.855,27267.959,27064.617
17318,55.0,99.0,NaN,2930.000000,279.634,311.462,220.524,177.329,247.806,215.977,...,25069.424,24499.764,24510.574,25402.479,25484.291,25770.318,26184.209,26663.855,26993.932,27359.191
17319,55.0,99.0,NaN,2930.250000,302.368,343.290,202.337,211.431,263.720,204.610,...,25523.170,24521.369,25023.805,25407.449,25412.559,25650.523,25960.604,26475.588,27386.002,27399.545


# Filtering 
# we will save "as is" (if there is no spectra we will delete that line from df)

In [16]:
kudmal_df.drop(['scan_time', 'time'], axis=1, inplace = True)
kudmal_df

C:\Anaconda3\envs\rebri_env\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,x,y,1922.31,1929.96,1937.62,1945.27,1952.92,1960.58,1968.23,1975.8799999999999,...,9495.41,9502.65,9509.89,9517.13,9524.37,9531.609999999999,9538.85,9546.09,9553.33,9560.57
0,0.0,0.0,25.008,50.016,-22.734,43.195,75.024,-6.820,6.820,-15.914,...,1049.286,675.148,793.172,999.197,832.101,916.913,1019.641,993.874,1197.294,1000.749
1,0.0,0.0,36.375,29.555,-13.641,13.641,40.922,-9.094,2.273,4.547,...,918.834,626.537,824.277,1103.591,846.448,949.166,1055.418,1072.684,1184.647,920.044
2,0.0,0.0,43.195,15.914,0.000,61.383,70.477,-20.461,-25.008,-27.281,...,1020.927,675.148,694.674,1063.822,803.408,972.204,974.920,928.200,1142.489,1016.890
3,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,1.0,63.657,40.922,-11.367,20.461,93.211,-43.195,-20.461,-50.016,...,1003.911,680.549,839.829,1053.880,784.279,981.419,1162.749,1103.332,1142.489,1028.996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17316,55.0,99.0,325.103,302.368,256.900,188.696,256.900,190.970,275.087,250.079,...,24950.316,24478.160,24697.203,25407.449,25422.121,25033.104,25772.773,26151.592,27107.760,26774.076
17317,55.0,99.0,325.103,302.368,222.798,263.720,263.720,213.704,245.532,270.540,...,25347.344,24548.375,24852.727,25462.133,25527.330,25774.928,25813.023,26663.855,27267.959,27064.617
17318,55.0,99.0,279.634,311.462,220.524,177.329,247.806,215.977,256.900,275.087,...,25069.424,24499.764,24510.574,25402.479,25484.291,25770.318,26184.209,26663.855,26993.932,27359.191
17319,55.0,99.0,302.368,343.290,202.337,211.431,263.720,204.610,254.626,227.345,...,25523.170,24521.369,25023.805,25407.449,25412.559,25650.523,25960.604,26475.588,27386.002,27399.545


In [17]:
# kudmal_df.drop_duplicates(keep = "first", inplace=True) # does not work, look at (x,y) = (0,0) and (0.1), we should take second spectra
kudmal_df.dropna(inplace = True)
kudmal_df

C:\Anaconda3\envs\rebri_env\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,x,y,1922.31,1929.96,1937.62,1945.27,1952.92,1960.58,1968.23,1975.8799999999999,...,9495.41,9502.65,9509.89,9517.13,9524.37,9531.609999999999,9538.85,9546.09,9553.33,9560.57
0,0.0,0.0,25.008,50.016,-22.734,43.195,75.024,-6.820,6.820,-15.914,...,1049.286,675.148,793.172,999.197,832.101,916.913,1019.641,993.874,1197.294,1000.749
1,0.0,0.0,36.375,29.555,-13.641,13.641,40.922,-9.094,2.273,4.547,...,918.834,626.537,824.277,1103.591,846.448,949.166,1055.418,1072.684,1184.647,920.044
2,0.0,0.0,43.195,15.914,0.000,61.383,70.477,-20.461,-25.008,-27.281,...,1020.927,675.148,694.674,1063.822,803.408,972.204,974.920,928.200,1142.489,1016.890
4,0.0,1.0,63.657,40.922,-11.367,20.461,93.211,-43.195,-20.461,-50.016,...,1003.911,680.549,839.829,1053.880,784.279,981.419,1162.749,1103.332,1142.489,1028.996
5,0.0,1.0,72.750,4.547,13.641,18.188,65.930,27.281,-43.195,-22.734,...,1106.004,707.555,813.909,1073.764,894.270,962.989,1104.611,1055.171,1323.769,1033.031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17316,55.0,99.0,325.103,302.368,256.900,188.696,256.900,190.970,275.087,250.079,...,24950.316,24478.160,24697.203,25407.449,25422.121,25033.104,25772.773,26151.592,27107.760,26774.076
17317,55.0,99.0,325.103,302.368,222.798,263.720,263.720,213.704,245.532,270.540,...,25347.344,24548.375,24852.727,25462.133,25527.330,25774.928,25813.023,26663.855,27267.959,27064.617
17318,55.0,99.0,279.634,311.462,220.524,177.329,247.806,215.977,256.900,275.087,...,25069.424,24499.764,24510.574,25402.479,25484.291,25770.318,26184.209,26663.855,26993.932,27359.191
17319,55.0,99.0,302.368,343.290,202.337,211.431,263.720,204.610,254.626,227.345,...,25523.170,24521.369,25023.805,25407.449,25412.559,25650.523,25960.604,26475.588,27386.002,27399.545


In [18]:
kudmal_df.reset_index(drop=True, inplace = True)

<div class="alert alert-block alert-info"> Last {x,y} point is highly represented. There too many <i>wl</i> at the same {104, 64} scanning stop point
</div>


In [19]:
kudmal_df.to_pickle("xy_wl"+my_fname+".pkl")

#df = pd.read_pickle("datasets/xy_wl__"+raw_Mal[4:18]+".pkl")
#df